In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [2]:
class GroupedConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, n_groups, stride):
        super().__init__()
        self.group_channels = in_channels // n_groups
        self.grouped_conv_paths = nn.ModuleList([
            nn.Conv2d(self.group_channels, out_channels // n_groups, kernel_size=3, stride=stride, padding=1,
                      bias=False) for _ in range(n_groups)
        ])
    
    def forward(self, x):
        x = torch.concat([
            path(x[:, self.group_channels * i:self.group_channels * (i + 1), ...]) for i, path in
            enumerate(self.grouped_conv_paths)], dim=1)
        return x


class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.depthwise_conv = nn.Sequential(
            # GroupedConvolution(in_channels, in_channels, in_channels, stride),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, bias=False,
                      groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.ReLU6(),
        )
        self.seeparable_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
    
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.seeparable_conv(x)
        return x

In [3]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride=1):
        super().__init__()
        self.expansion_factor = expansion_factor
        self.use_skip_connection = (stride == 1 and in_channels == out_channels)
        self.bottleneck = nn.Sequential(
            nn.Sequential(
                nn.Conv2d(in_channels, in_channels * expansion_factor, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(in_channels * expansion_factor),
                nn.ReLU6(),
            ) if self.expansion_factor != 1 else nn.Identity(),
            DepthwiseSeparableConvolution(in_channels * expansion_factor, out_channels, stride)
        )
    
    def forward(self, x):
        bottleneck = self.bottleneck(x)
        bottleneck = bottleneck + x if self.use_skip_connection else bottleneck
        return bottleneck


x_batch = torch.randn(32, 64, 52, 52)
print(InvertedResidualBlock(64, 32, 6, 2)(x_batch).shape)

torch.Size([32, 32, 26, 26])


In [4]:
class MobileNetV2(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU6()
        )
        self.conv2 = InvertedResidualBlock(32, 16, 1, 1)
        self.conv3 = self.make_layers(16, 24, 6, 2, 2)
        self.conv4 = self.make_layers(24, 32, 6, 3, 2)
        self.conv5 = self.make_layers(32, 64, 6, 4, 2)
        self.conv6 = self.make_layers(64, 96, 6, 3, 1)
        self.conv7 = self.make_layers(96, 160, 6, 3, 2)
        self.conv8 = self.make_layers(160, 320, 6, 1, 1)
        self.conv9 = nn.Sequential(
            nn.Conv2d(320, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.ReLU6()
        )
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(1280, n_classes),
        )
    
    def make_layers(self, in_channels, out_channels, expansion_factor, n_blocks, stride):
        layers = [InvertedResidualBlock(in_channels, out_channels, expansion_factor, stride)]
        for layer in range(n_blocks - 1):
            layers.append(InvertedResidualBlock(out_channels, out_channels, expansion_factor))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x


x_batch = torch.randn(32, 3, 224, 224)
print(MobileNetV2(3, 1000)(x_batch).shape)

torch.Size([32, 1000])


In [5]:
mobilenet_v2_model = MobileNetV2(3, 1000)
summary(mobilenet_v2_model, input_size=(1, 3, 224, 224), col_names=['num_params', 'mult_adds'], device='cpu', depth=2)

Layer (type:depth-idx)                                       Param #                   Mult-Adds
MobileNetV2                                                  --                        --
├─Sequential: 1-1                                            --                        --
│    └─Conv2d: 2-1                                           864                       10,838,016
│    └─BatchNorm2d: 2-2                                      64                        64
│    └─ReLU6: 2-3                                            --                        --
├─InvertedResidualBlock: 1-2                                 --                        --
│    └─Sequential: 2-4                                       896                       10,035,296
├─Sequential: 1-3                                            --                        --
│    └─InvertedResidualBlock: 2-5                            5,136                     29,202,864
│    └─InvertedResidualBlock: 2-6                            8,832   

In [6]:
mobilenet_v2_torch_model = models.mobilenet_v2()
summary(mobilenet_v2_torch_model, input_size=(1, 3, 224, 224),
        col_names=['num_params', 'mult_adds'], device='cpu', depth=2)

Layer (type:depth-idx)                             Param #                   Mult-Adds
MobileNetV2                                        --                        --
├─Sequential: 1-1                                  --                        --
│    └─Conv2dNormActivation: 2-1                   928                       10,838,080
│    └─InvertedResidual: 2-2                       896                       10,035,296
│    └─InvertedResidual: 2-3                       5,136                     29,202,864
│    └─InvertedResidual: 2-4                       8,832                     25,740,912
│    └─InvertedResidual: 2-5                       10,000                    15,467,392
│    └─InvertedResidual: 2-6                       14,848                    10,989,376
│    └─InvertedResidual: 2-7                       14,848                    10,989,376
│    └─InvertedResidual: 2-8                       21,056                    7,564,928
│    └─InvertedResidual: 2-9                      

## Training

In [7]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_ds = datasets.CIFAR100(data_dir, train=True, download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.CIFAR100(data_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import wandb
from src.engine import *

config = dict(batch_size=64, lr=3e-4, epochs=20, dataset='CIFAR100')
with wandb.init(project='pytorch-study', name='MobileNetV2', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    mobilenet_v2_model = MobileNetV2(3, n_classes).to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mobilenet_v2_model.parameters(), lr=w_config.lr)
    
    loss_history, acc_history = train(mobilenet_v2_model, train_dl, val_dl, criterion, optimizer, w_config.epochs,
                                      DEVICE, run) 

Epoch=18:  85%|████████▌ | 17/20 [39:06<07:27, 149.04s/it, train_loss=0.383, train_acc=87.64%, val_loss=2.523, val_acc=48.65%]